# Exercícios Pandas Parte 2 # 

In [37]:
import pandas as pd
import numpy as np


## Exercício 1 ##
Olhando apenas para o grupo de divorciados, qual o percentual deles que é empreendedor (entrepreneur)? E estudantes?.</p>


In [12]:
bank_df= pd.read_csv('bank-full.csv', sep=';')
bank_df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


<h2>Resolução</h2>

<p> Primeiro, vou verificar quantos tipos únicos temos de jobs e estados cívis.</p>

In [15]:
#verificando tipos únicos
bank_df.job.unique()
bank_df.marital.unique()

array(['married', 'single', 'divorced'], dtype=object)

<p> Para job temos as seguintes categorias: </p>
<ul> management', 'technician', 'entrepreneur', 'blue-collar',
       'unknown', 'retired', 'admin.', 'services', 'self-employed',
       'unemployed', 'housemaid', 'student </ul>
<p> E para marital temos as seguintes categorias:
<ul>['married', 'single', 'divorced'] </ul>

In [35]:
#filtrando divorciados e estudantes que sao divorciados
quantidade= bank_df.query('job in[ "entrepreneur", "student"] and marital == "divorced"')

#contando a quantidade de pessoas que são divorciadas e ou estudantes ou empresarios
qtd= quantidade.shape[0]

#quantidade de divorciados
divorciados= bank_df.query('marital == "divorced"')
qtde_divorciados= divorciados.shape[0]

#percentual de divorciados que são estudantes ou empresários
percentual= (qtd/qtde_divorciados) *100

print(f'O número de pessoas que são divorciadas e são ou estudante ou empresários é de {qtd}')
print(f' O total dos divorciados é de {qtde_divorciados}')
print(f' E o percentual de divorciados que são divorciados e ao mesmo tempo empresários ou estudantes é de {percentual:.2f} %')

#obtendo o percentual

O número de pessoas que são divorciadas e são ou estudante ou empresários é de 185
 O total dos divorciados é de 5207
 E o percentual de divorciados que são divorciados e ao mesmo tempo empresários ou estudantes é de 3.55 %


Talvez tenha melhor forma de fazer isso, mas seguimos em frente. O importante é continuar.
</details>

## Exercício 2 ##
 O seu chefe decidiu criar duas categorias, Premium e Gold, para focar nos clientes mais promissores. Para a categoria Premium, entrarão os que possuem saldo (balance) nos top 10%. Ou seja, o grupo acima do percentil 90. Já o grupo Gold, ficarão os que possuem saldo nos top 20%, mas abaixo do grupo Premium. Ou seja, do p80 até o último do p90. Verifique quantos clientes possuem no grupo Premium e no Gold.

Primeiramente, iremos calcular o percentil de 90% e 80% do balance:

grupo premium: 10 % com mais balance, percentil de 90% 
grupo gold: percentil de 80% até 90%

categoria defina pelo balance

In [56]:
# definindo percentil de 80%
percent_80= bank_df['balance'].quantile(0.80)

#definindo percentil de 90%
percent_90= bank_df['balance'].quantile(0.90)

percent_80
percent_90

#definindo intervalos para o cut

bins= [bank_df['balance'].min(), percent_80, percent_90, bank_df['balance'].max()]
labels=['gold', 'premium', 'top 10%']

#categorizando os dados

bank_df['categoria']=pd.cut(bank_df['balance'], bins=bins, labels=labels, include_lowest= True)

#contando quantos clientes estão em cada grupo
contagem= bank_df['categoria'].value_counts()

print(bank_df)
print(f"O número de clientes na categoria Gold : {contagem.get('gold',0)}")
print(f"O número de clientes na categoria Premium: {contagem.get('premium',0)}")

       age           job   marital  education default  balance housing loan  \
0       58    management   married   tertiary      no     2143     yes   no   
1       44    technician    single  secondary      no       29     yes   no   
2       33  entrepreneur   married  secondary      no        2     yes  yes   
3       47   blue-collar   married    unknown      no     1506     yes   no   
4       33       unknown    single    unknown      no        1      no   no   
...    ...           ...       ...        ...     ...      ...     ...  ...   
45206   51    technician   married   tertiary      no      825      no   no   
45207   71       retired  divorced    primary      no     1729      no   no   
45208   72       retired   married  secondary      no     5715      no   no   
45209   57   blue-collar   married  secondary      no      668      no   no   
45210   37  entrepreneur   married  secondary      no     2971      no   no   

         contact  day month  duration  campaign  pd